In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import scipy.stats as stat
import pylab 
import math
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression as LGR,Lasso
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold

In [10]:
Fe_CAAC_dataset = df = pd.read_excel("F:\\Nitrogen_activation\\rajaraman\\office\\ofifice\\jupyter\\Extension\\T07\\heatmap\\agostic\\Ridge\\N2_activation_extension_T07_heatmap_agostaic_descrptors.xlsx")

In [3]:
df_1=df.drop(['F00','T01','T02','T03',],axis=1)
df_2=df_1.drop([0,18,9])
df_3=df_1.drop([0,18,9,8,26,17])
df_4=df_1.drop([8,26,17])

In [4]:
X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=20)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)
# transform train and test sets
X_train_scaled_MM = scaler.transform(X_train)
X_test_scaled_MM = scaler.transform(X_test)
X_train_scaled_MM = pd.DataFrame(X_train_scaled_MM, columns=X_train.columns)
X_test_scaled_MM = pd.DataFrame(X_test_scaled_MM, columns=X_test.columns)

# Define the parameter grid
param_grid = {
    'alpha': np.logspace(-4, 4, 100)  # Testing a range of alpha values from 0.0001 to 100
}


# Initialize a Lasso model
ridge_reg_model = Ridge()

# Use GridSearchCV to find the best alpha
grid_search = GridSearchCV(ridge_reg_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled_MM, y_train)

# Best alpha value found by GridSearchCV
best_alpha = grid_search.best_params_['alpha']
print(f"Best alpha value found by GridSearchCV: {best_alpha}")

Best alpha value found by GridSearchCV: 0.03853528593710531


In [5]:
X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=20)


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)
# transform train and test sets
X_train_scaled_MM = scaler.transform(X_train)
X_test_scaled_MM = scaler.transform(X_test)
X_train_scaled_MM = pd.DataFrame(X_train_scaled_MM, columns=X_train.columns)
X_test_scaled_MM = pd.DataFrame(X_test_scaled_MM, columns=X_test.columns)



ridge_reg_model = Ridge(alpha=0.03)
ridge_reg_model.fit(X_train_scaled_MM,y_train)
# prediction on Training data
training_data_prediction_X_train_ridge_reg = ridge_reg_model.predict(X_train_scaled_MM)
# prediction on Test data
test_data_prediction_X_test_ridge_reg = ridge_reg_model.predict(X_test_scaled_MM)

# R squared Error
error_score_r2 = metrics.r2_score(y_train, training_data_prediction_X_train_ridge_reg)
error_score_MAE = metrics.mean_absolute_error(y_train, training_data_prediction_X_train_ridge_reg)
error_score_MSE = metrics.mean_squared_error(y_train, training_data_prediction_X_train_ridge_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_train,training_data_prediction_X_train_ridge_reg)))
# R squared Error
error_score_r2 = metrics.r2_score(y_test, test_data_prediction_X_test_ridge_reg)
error_score_MAE = metrics.mean_absolute_error(y_test, test_data_prediction_X_test_ridge_reg)
error_score_MSE = metrics.mean_squared_error(y_test, test_data_prediction_X_test_ridge_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_test,test_data_prediction_X_test_ridge_reg)))

coeff = ridge_reg_model.coef_
print(coeff)
features = X_train.columns
features = X_train.columns
df_coeff = pd.DataFrame({"features":features,"coeff":coeff})
df_coeff.sort_values("coeff")


R squared Error :  0.9184606617524969
mean_absolute_error :  8.844089367596712
mean_squared_error :  142.58120210812578
RMSE 11.940737083954481
R squared Error :  0.8100566107432629
mean_absolute_error :  10.452443427618736
mean_squared_error :  134.48056657955772
RMSE 11.596575640229219
[-121.48643163   34.28778937]


,features,coeff
0,F03,-121.486432
1,F08,34.287789


In [6]:

# Initialize the Leave-One-Out cross-validator
loo = LeaveOneOut()

X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model

# Apply MinMax Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)  # Scale the features and store them in X_scaled


# Initialize lists to store predictions and actual values
y_pred_loo = []
y_true_loo = []

# Loop over each training/test split
for train_index, test_index in loo.split(X_scaled):
    X_train_loo, X_test_loo = X_scaled[train_index], X_scaled[test_index]
    y_train_loo, y_test_loo = y.iloc[train_index], y.iloc[test_index]
    
    # Train the Lasso model
    ridge_reg_model = Ridge(alpha=0.03)
    ridge_reg_model.fit(X_train_loo, y_train_loo)
    
    # Predict the left-out sample
    y_pred_loo.append(ridge_reg_model.predict(X_test_loo)[0])
    y_true_loo.append(y_test_loo.values[0])

# Convert lists to arrays
y_pred_loo = np.array(y_pred_loo)
y_true_loo = np.array(y_true_loo)

# Calculate LOO performance metrics
Q2_loo = r2_score(y_true_loo, y_pred_loo)
MAE_loo = mean_absolute_error(y_true_loo, y_pred_loo)
MSE_loo = mean_squared_error(y_true_loo, y_pred_loo)
RMSE_loo = np.sqrt(MSE_loo)

# Print the results
print("LOO Q^2 score: ", Q2_loo)
print("LOO Mean Absolute Error: ", MAE_loo)
print("LOO Mean Squared Error: ", MSE_loo)
print("LOO Root Mean Squared Error: ", RMSE_loo)



LOO Q^2 score:  0.871714032211474
LOO Mean Absolute Error:  10.695559288749834
LOO Mean Squared Error:  192.54144369426473
LOO Root Mean Squared Error:  13.875930372204406


In [7]:
# can tune other metrics, such as MSE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X)
# transform train and test sets
X_scaled_MM = scaler.transform(X)
X_scaled_MM = pd.DataFrame(X_scaled_MM, columns=X.columns)

scores = cross_val_score(ridge_reg_model, X_scaled_MM, y, cv=2)
print(scores)
print(np.mean(scores))


scores = cross_val_score(ridge_reg_model, X_scaled_MM, y, cv=3)
print(scores)
print(np.mean(scores))

scores = cross_val_score(ridge_reg_model, X_scaled_MM, y, cv=4)
print(scores)
print(np.mean(scores))

scores = cross_val_score(ridge_reg_model, X_scaled_MM, y, cv=5)
print(scores)
print(np.mean(scores))



[0.85793688 0.89294195]
0.8754394147359617
[0.89542174 0.8741468  0.88474633]
0.8847716218715685
[0.80840124 0.83257931 0.96819693 0.84347103]
0.8631621303726997
[0.69200317 0.9282537  0.64523604 0.9710031  0.77407384]
0.8021139704757125


In [8]:
coeff = ridge_reg_model.coef_
features = X_train.columns
features = X_train.columns
df_coeff = pd.DataFrame({"features":features,"coeff":coeff})
df_coeff.sort_values("coeff")

,features,coeff
0,F03,-122.187217
1,F08,33.351960


In [9]:
comparison_df_train_ridge = pd.DataFrame({'Actual Barrier (kJ/mol)': y_train, 'Predicted Barrier (kJ/mol)': training_data_prediction_X_train_ridge_reg})

comparison_df_train_ridge.to_excel('predicted_vs_actual_comparison_train_ridge.xlsx', index=False)

comparison_df_test_ridge = pd.DataFrame({'Actual Barrier (kJ/mol)': y_test, 'Predicted Barrier (kJ/mol)': test_data_prediction_X_test_ridge_reg})

comparison_df_test_ridge.to_excel('predicted_vs_actual_comparison_test_ridge.xlsx', index=False)

print(comparison_df_train_ridge)

print(comparison_df_test_ridge)

    Actual Barrier (kJ/mol)  Predicted Barrier (kJ/mol)
15               134.989547                  133.561368
16               172.156151                  162.255012
22                60.127639                   84.883152
19                48.921745                   49.471220
25               153.204294                  156.895449
13               111.384753                   88.256245
6                126.165031                  132.146005
11                84.663254                   69.836381
7                158.160792                  160.493288
1                 25.243201                   49.260021
3                 63.400511                   67.193347
10                54.848496                   49.084439
14                96.725090                  102.358536
12                77.707753                   76.133297
20                70.614886                   63.681935
4                 97.057793                   89.861240
    Actual Barrier (kJ/mol)  Predicted Barrier (